In [ ]:
import copy
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import GroupKFold,GroupShuffleSplit
from torch.utils.data import Dataset
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader, Subset

from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import trange
from time import time
root_dir = Path('/kaggle/input/osic-pulmonary-fibrosis-progression')
model_dir = Path('/kaggle/working')

# 要写自定义的数据集（__init__,__len__,__getitem__）

In [ ]:
# pd.read_csv(Path(root_dir)/"train.csv").head(10)

In [ ]:
# #先做些basic check
# tr = pd.read_csv(Path(root_dir)/"train.csv")# print(len(tr))=1549
# tr[tr.duplicated(keep=False,subset=['Patient', 'Weeks'])==True]#把重复的打印了一下康康
# # 好神奇啊，这是一周内看了两次病？

In [ ]:
# sub = pd.read_csv(Path(root_dir)/"sample_submission.csv")
# sub #预测的时候是对样本做全段预测   5个病人*146周

In [ ]:
# tst = pd.read_csv(Path(root_dir)/"test.csv")#test_data
# tst#已知 week0 CT info + tabular below

In [ ]:
#这部分对sub的结构小小处理了一下，到时候做test的时候再说
# sub = pd.read_csv(Path(root_dir)/"sample_submission.csv")
# sub['Patient'] = sub['Patient_Week'].apply(lambda x: x.split('_')[0])
# sub['Weeks'] = sub['Patient_Week'].apply(lambda x: int(x.split('_')[-1]))
# sub = sub[['Patient', 'Weeks', 'Confidence', 'Patient_Week']]
# sub.merge(tst.drop('Weeks', axis=1), on="Patient",how='inner')

In [ ]:
# tr = pd.read_csv(Path(root_dir)/"train.csv")#train_data
# tr.drop_duplicates(keep="first", inplace=True, subset=['Patient', 'Weeks'])
# tr['minweek']=tr.groupby('Patient')['Weeks'].transform(min)

In [ ]:
# tr.iloc[tr['Percent'].argmax(),:]#这个percent还是可以用一下的，不过我看的两个本子好像都没用

In [ ]:
# base = tr.loc[tr.Weeks == tr.minweek]
# base = base[['Patient', 'FVC']].copy()
# base.columns = ['Patient', 'minweek_FVC']
# tr=tr.merge(base, on='Patient', how='inner')#train部分数据不脏，left和inner都没问题
# tr['base_week']=tr['Weeks']-tr['minweek']

In [ ]:
# base#恰好对应176个人

In [ ]:
# data=tr.copy()
# COLS = ['Sex', 'SmokingStatus']
# FE = []
# for col in COLS:
#     for mod in data[col].unique():
#         FE.append(mod)
#         data[mod] = (data[col] == mod).astype(int)

In [ ]:
# import numpy as np
# x = np.array([0, 1, 2, 3])
# y = np.array([-1, 0.2, 0.9, 2.1])
# A = np.vstack([x, np.ones(len(x))]).T

In [ ]:
# np.vstack([x, np.ones(len(x))])

In [ ]:
# np.linalg.lstsq(A,y)

In [ ]:
# import pydicom
# import cv2
# def get_img(path):
#     d = pydicom.dcmread(path)
#     return cv2.resize((d.pixel_array - d.RescaleIntercept) / (d.RescaleSlope), (512, 512))

# d = pydicom.dcmread(root_dir/'train/ID00007637202177411956430'/'6.dcm')
# print(d.pixel_array)
# print(d.RescaleIntercept)
# print(d.RescaleSlope)
# im=((d.pixel_array - d.RescaleIntercept) / (d.RescaleSlope * 1000))

# # %matplotlib inline
# import os
# from matplotlib import pyplot as plt
# k=0
# for i in os.listdir(root_dir/'train/ID00007637202177411956430'):
#     if k>=0:
#         m=get_img(root_dir/'train/ID00007637202177411956430'/i)
# #         print(m)
#         plt.imshow(m,cmap='g')
#         plt.show()
#         k=k+1
# #         plt.show()


In [ ]:
#819 new！！
# tst = pd.read_csv(Path(root_dir)/"test.csv")#test_data
# tst['minweek']=tst['Weeks']
# tst['minweek_FVC']=tst['FVC']
# sub = pd.read_csv(Path(root_dir)/"sample_submission.csv")
# sub['Patient'] = sub['Patient_Week'].apply(lambda x: x.split('_')[0])
# sub['Weeks'] = sub['Patient_Week'].apply(lambda x: int(x.split('_')[-1]))
# sub = sub[['Patient', 'Weeks', 'Confidence', 'Patient_Week']]
# sub = sub.merge(tst.drop('Weeks', axis=1), on="Patient",how="inner")#样子就是之前单元格中的样子
# sub['base_week']=sub['Weeks']-sub['minweek']

# tr = pd.read_csv(Path(root_dir)/"train.csv")#train_data
# tr.drop_duplicates(keep="first", inplace=True, subset=['Patient', 'Weeks']) #暂且留其中一个把 keep=“first”
# tst = pd.read_csv(Path(root_dir)/"test.csv")#test_data
# tr['minweek']=tr.groupby('Patient')['Weeks'].transform(min)
# base = tr.loc[tr.Weeks == tr.minweek] #找基准
# base = base[['Patient', 'FVC']]
# base.columns = ['Patient', 'minweek_FVC']#改名
# tr=tr.merge(base, on='Patient', how='inner')#train部分数据不脏，left和inner都没问题
# tr['base_week']=tr['Weeks']-tr['minweek']
# data=tr.append(sub)

In [ ]:
# data[~data['Patient_Week'].isna()]

# 从这里开始

In [ ]:
class ClinicalDataset(Dataset):
    def __init__(self, root_dir, mode, transform=None):
        self.transform = transform   #tabular的东西transform先挂着，感觉一般不用
        self.mode = mode             #train or test
        tst = pd.read_csv(Path(root_dir)/"test.csv")#test_data
        tst['minweek']=tst['Weeks']
        tst['minweek_FVC']=tst['FVC']
        sub = pd.read_csv(Path(root_dir)/"sample_submission.csv")
        sub['Patient'] = sub['Patient_Week'].apply(lambda x: x.split('_')[0])
        sub['Weeks'] = sub['Patient_Week'].apply(lambda x: int(x.split('_')[-1]))
        sub = sub[['Patient', 'Weeks', 'Confidence', 'Patient_Week']]
        sub = sub.merge(tst.drop('Weeks', axis=1), on="Patient",how="inner")#样子就是之前单元格中的样子
        sub['base_week']=sub['Weeks']-sub['minweek']

        tr = pd.read_csv(Path(root_dir)/"train.csv")#train_data
        tr.drop_duplicates(keep="first", inplace=True, subset=['Patient', 'Weeks']) #暂且留其中一个把 keep=“first”
        tst = pd.read_csv(Path(root_dir)/"test.csv")#test_data
        tr['minweek']=tr.groupby('Patient')['Weeks'].transform(min)
        base = tr.loc[tr.Weeks == tr.minweek] #找基准
        base = base[['Patient', 'FVC']]
        base.columns = ['Patient', 'minweek_FVC']#改名
        tr=tr.merge(base, on='Patient', how='inner')#train部分数据不脏，left和inner都没问题
        tr['base_week']=tr['Weeks']-tr['minweek']
        data=tr.append(sub)
        #归一标准化，数据分位
        COLS = ['Sex', 'SmokingStatus']
        self.FE = []
        for col in COLS:
            for mod in data[col].unique():
                self.FE.append(mod)
                data[mod] = (data[col] == mod).astype(int)
    
        data['N_age'] = (data['Age'] - data['Age'].min()) / \
                      (data['Age'].max() - data['Age'].min())
        data['N_base_minweek_FVC'] = (data['minweek_FVC'] - data['minweek_FVC'].min()) / \
                       (data['minweek_FVC'].max() - data['minweek_FVC'].min())
        
        data['N_week'] = (data['base_week'] - data['base_week'].min()) / \
                       (data['base_week'].max() - data['base_week'].min())
        
        data['N_percent'] = (data['Percent'] - data['Percent'].min()) / \
                          (data['Percent'].max() - data['Percent'].min())
        
        self.FE += ['N_age', 'N_percent', 'N_week', 'N_base_minweek_FVC']
        
        #目前这个pytorchDatasets 只针对train
        if self.mode=='train':
            self.raw = data[data['Patient_Week'].isna()].reset_index()
        elif self.mode=='test':
            self.raw =data[~data['Patient_Week'].isna()].reset_index()
        
        del base
        del data

    def __len__(self):
        return len(self.raw)

    def __getitem__(self, idx):
        #为了dataframe的切片，转换一下
        if torch.is_tensor(idx):
            idx = idx.tolist()

        sample = {
            'patient_id': self.raw['Patient'].iloc[idx],
            'features': self.raw[self.FE].iloc[idx].values,
            'target': self.raw['FVC'].iloc[idx]
        }
        if self.transform:
            sample = self.transform(sample)

        return sample
    
    def group_kfold(self, n_splits):
        gkf = GroupKFold(n_splits=n_splits)
        groups = self.raw['Patient']
        for train_idx, val_idx in gkf.split(self.raw, self.raw, groups):
            train = Subset(self, train_idx)
            val = Subset(self, val_idx)
            yield train, val
    def group_split(self, test_size=0.1):
        gss = GroupShuffleSplit(n_splits=1, test_size=test_size)
        groups = self.raw['Patient']
        idx = list(gss.split(self.raw, self.raw, groups))
        train = Subset(self, idx[0][0])
        val = Subset(self, idx[0][1])
        return train, val

In [ ]:
# myDataSet=ClinicalDataset(root_dir, "test")
# i=0
# for k in enumerate(myDataSet):
#     if i<10:
#         print(k)
#         i=i+1

In [ ]:
# import torch
# torch.max(torch.FloatTensor([1,2,3,4,2,8]),torch.FloatTensor([1,2,3,4,2,9])).unsqueeze(1)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class QuantModel(nn.Module):
    def __init__(self, in_tabular_features=9, out_quantiles=3):
        super(QuantModel, self).__init__()
        self.fc1 = nn.Linear(in_tabular_features, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 50)
        self.fc4 = nn.Linear(50, 3)
        self.fc5 = nn.Linear(3,out_quantiles)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        x = self.fc5(x)
        return x

#分位数回归
quantiles = (0.2, 0.5, 0.8)
def quantile_loss(preds, target, quantiles):
    assert not target.requires_grad
    assert preds.size(0) == target.size(0)
    losses = []
    for i, q in enumerate(quantiles):
        errors = target - preds[:, i]
        losses.append(torch.max((q - 1) * errors, q * errors).unsqueeze(1))#添加维度   （x,1）
    loss = torch.mean(torch.sum(torch.cat(losses, dim=1), dim=1))
    return loss

In [ ]:
# ten=torch.FloatTensor([[1.        , 0.        , 1.        , 0.        , 0.        ,
#        0.76923077, 0.23639327, 0.        , 0.24145617],[1.        , 0.        , 1.        , 0.        , 0.        ,
#        0.76923077, 0.23639327, 0.        , 0.24145617]])

In [ ]:
# net=QuantModel()

In [ ]:
# losses=[]
# losses.append(net(ten))

In [ ]:
# losses

In [ ]:
# net(ten).size()

In [ ]:
# torch.mean((torch.sum(torch.cat(losses,dim=1),dim=1)))

In [ ]:
# Path(model_dir).mkdir(parents=True, exist_ok=True)
class Monitor:
    def __init__(self, model, es_patience, experiment_name, tensorboard_dir,
                 num_epochs, dataset_sizes, model_file):

        self.model = model
        self.model_file = model_file
        self.es_patience = es_patience
        self.tensorboard_dir = tensorboard_dir
        self.dataset_sizes = dataset_sizes
        date_time = datetime.now().strftime("%Y%m%d-%H%M")
        log_dir = tensorboard_dir / f'{experiment_name}-{date_time}'
        self.w = SummaryWriter(log_dir)

        self.bar = trange(num_epochs, desc=experiment_name)

        self.epoch_loss = {'train': np.inf, 'val': np.inf}
        self.epoch_metric = {'train': -np.inf, 'val': -np.inf}
        self.best_loss = np.inf
        self.best_model_wts = None

        self.e = {'train': 0, 'val': 0}  # epoch counter
        self.t = {'train': 0, 'val': 0}  # global time-step (never resets)
        self.running_loss = 0.0
        self.running_metric = 0.0
        self.es_counter = 0

    def reset_epoch(self):
        self.running_loss = 0.0
        self.running_metric = 0.0

    def step(self, loss, inputs, preds, targets, phase):
        self.running_loss += loss.item() * inputs.size(0)
        self.running_metric += self.metric(preds, targets).sum()
        self.t[phase] += 1

    def log_epoch(self, phase):
        self.epoch_loss[phase] = self.running_loss / self.dataset_sizes[phase]
        self.epoch_metric[phase] = self.running_metric / self.dataset_sizes[phase]
        self.bar.set_postfix(
            a_train_loss=f'{self.epoch_loss["train"]:0.1f}',
            b_val_loss=f'{self.epoch_loss["val"]:0.1f}',
            c_train_metric=f'{self.epoch_metric["train"]:0.4f}',
            d_val_metric=f'{self.epoch_metric["val"]:0.4f}',
            es_counter=self.es_counter
        )
        self.w.add_scalar(
            f'Loss/{phase}', self.epoch_loss[phase], self.e[phase])
        self.w.add_scalar(
            f'Accuracy/{phase}', self.epoch_metric[phase], self.e[phase])

        self.e[phase] += 1

        # Early stop and model backup
        early_stop = False
        if phase == 'val':
            if self.epoch_loss['val'] < self.best_loss:
                self.best_loss = self.epoch_loss['val']
                self.best_model_wts = copy.deepcopy(self.model.state_dict())
                torch.save(self.best_model_wts, self.model_file)
                self.es_counter = 0
            else:
                self.es_counter += 1
                if self.es_counter >= self.es_patience:
                    early_stop = True
                    self.bar.close()

        return early_stop

    @staticmethod
    def metric(preds, targets):
        sigma = preds[:, 2] - preds[:, 0]
        sigma[sigma < 70] = 70
        delta = (preds[:, 1] - targets).abs()
        delta[delta > 1000] = 1000
        return -np.sqrt(2) * delta / sigma - torch.log(np.sqrt(2) * sigma)

In [ ]:
models = []
num_kfolds=10
# Load the data
mydata = ClinicalDataset(root_dir=root_dir, mode='train')
folds = mydata.group_kfold(num_kfolds)
t0 = time()
batch_size = 16
learning_rate = 3e-3
num_epochs = 2000
es_patience = 20
quantiles = (0.25, 0.5, 0.75)
model_name ='Pacific'
tensorboard_dir = Path('/kaggle/working/runs')
trainset=mydata.group_split(0.15)[0]
valset=mydata.group_split(0.15)[1]

# for fold, (trainset, valset) in enumerate(folds):
    # Prepare to save model weights
fold=0
Path(model_dir).mkdir(parents=True, exist_ok=True)
now = datetime.now()
fname = f'{model_name}-{now.year}{now.month:02d}{now.day:02d}_{fold}.pth'
model_file = Path(model_dir) / fname

dataset_sizes = {'train': len(trainset), 'val': len(valset)}
dataloaders = {
    'train': DataLoader(trainset, batch_size=batch_size,
                        shuffle=True, num_workers=2),
    'val': DataLoader(valset, batch_size=batch_size,
                      shuffle=True, num_workers=2)
}

# Create the model and optimizer
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = QuantModel().to(device)
optimizer = Adam(model.parameters(), lr=learning_rate)
scheduler = StepLR(optimizer, step_size=20, gamma=0.5)
monitor = Monitor(
    model=model,
    es_patience=es_patience,
    experiment_name=f'{model_name}_fold_{fold}',
    tensorboard_dir=tensorboard_dir,
    num_epochs=num_epochs,
    dataset_sizes=dataset_sizes,
    model_file=model_file
)

# Training loop
for epoch in monitor.bar:
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()  # Set model to training mode
        else:
            model.eval()   # Set model to evaluate mode

        monitor.reset_epoch()

        # Iterate over data
        for batch in dataloaders[phase]:
            inputs = batch['features'].float().to(device)
            targets = batch['target'].to(device)

            # zero the parameter gradients
            optimizer.zero_grad()
            # forward
            # track gradients if only in train
            with torch.set_grad_enabled(phase == 'train'):
                preds = model(inputs)
                loss = quantile_loss(preds, targets, quantiles)
                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            monitor.step(loss, inputs, preds, targets, phase)

#             epoch statistics
        early_stop = monitor.log_epoch(phase)

    if early_stop:
        break

    # Updates the learning rate
    scheduler.step()

# load best model weights
model.load_state_dict(monitor.best_model_wts)
models.append(model)

print(f'Training complete! Time: {timedelta(seconds=time() - t0)}')

In [ ]:
# next(iter(dataloaders['train']))

In [ ]:
# len([4562, 3499, 2793, 2582, 3327, 1529, 2774, 4169, 3370, 4635, 2694, 2914,
#          1584, 1450, 2541, 2561, 1841, 1401, 2727, 3237, 1613, 2783, 2015, 2182,
#          4130, 2415, 3829, 3603, 3252, 2600, 3363, 2889])

In [ ]:
data = ClinicalDataset(root_dir, mode='test')
avg_preds = np.zeros((len(data), len(quantiles)))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
for model in models:
    dataloader = DataLoader(data, batch_size=batch_size,
                            shuffle=False, num_workers=2)
    preds = []
    for batch in dataloader:
        inputs = batch['features'].float().to(device)
        with torch.no_grad():
            x = model(inputs)
            preds.append(x)

    preds = torch.cat(preds, dim=0).cpu().numpy()
    avg_preds += preds

avg_preds /= len(models)
df = pd.DataFrame(data=avg_preds, columns=list(quantiles))
df['Patient_Week'] = data.raw['Patient_Week']
df['FVC'] = df[quantiles[1]]
df['Confidence'] = df[quantiles[2]] - df[quantiles[0]]
df = df.drop(columns=list(quantiles))
df.to_csv('submission.csv', index=False)

In [ ]:
# tr = pd.read_csv(Path(root_dir)/"train.csv")#train_data
# tr.drop_duplicates(keep="first", inplace=True, subset=['Patient', 'Weeks']) #暂且留其中一个把 keep=“first”
# tst = pd.read_csv(Path(root_dir)/"test.csv")#test_data

# #         sub = pd.read_csv(Path(root_dir)/"sample_submission.csv")
# #         sub['Patient'] = sub['Patient_Week'].apply(lambda x: x.split('_')[0])
# #         sub['Weeks'] = sub['Patient_Week'].apply(lambda x: int(x.split('_')[-1]))
# #         sub = sub[['Patient', 'Weeks', 'Confidence', 'Patient_Week']]
# #         sub = sub.merge(tst.drop('Weeks', axis=1), on="Patient",how="inner")#样子就是之前单元格中的样子
# #         sub['minweek']=sub.groupby('Patient')['Weeks'].transform(min)#待定

# tr['minweek']=tr.groupby('Patient')['Weeks'].transform(min)
# base = tr.loc[tr.Weeks == tr.minweek] #找基准
# base = base[['Patient', 'FVC']]
# base.columns = ['Patient', 'minweek_FVC']#改名
# tr=tr.merge(base, on='Patient', how='inner')#train部分数据不脏，left和inner都没问题
# del base
# tr['base_week']=tr['Weeks']-tr['minweek']
# #归一标准化，数据分位
# data=tr.copy()
# COLS = ['Sex', 'SmokingStatus']
# FE = []
# for col in COLS:
#     for mod in data[col].unique():
#         FE.append(mod)
#         data[mod] = (data[col] == mod).astype(int)

# data['N_age'] = (data['Age'] - data['Age'].min()) / \
#               (data['Age'].max() - data['Age'].min())
# data['N_base_minweek_FVC'] = (data['minweek_FVC'] - data['minweek_FVC'].min()) / \
#                (data['minweek_FVC'].max() - data['minweek_FVC'].min())

# data['N_week'] = (data['base_week'] - data['base_week'].min()) / \
#                (data['base_week'].max() - data['base_week'].min())

# data['N_percent'] = (data['Percent'] - data['Percent'].min()) / \
#                   (data['Percent'].max() - data['Percent'].min())

# FE += ['N_age', 'N_percent', 'N_week', 'N_base_minweek_FVC']

In [ ]:
# raw=data

In [ ]:
# raw

In [ ]:
# gkf=GroupKFold(10)

In [ ]:
# for i in gkf.split(raw,raw,raw['Patient']):
#     print(len(i[0]),len(i[1]))
#     train=i[0]
#     val=i[1]

In [ ]:
# from torch.utils.data import Subset

In [ ]:
# Subset(train)